In [29]:
import os
os.chdir("../../../")

In [30]:
import re

# Parsing Answers from LLaMa-2

In [31]:
import pandas as pd

In [32]:
INPUT_PATH = r"D:\Studying\UoR\1. Data Mining\Final_Project\data\generated\answer_russia.csv"

In [33]:
df = pd.read_csv(INPUT_PATH)
df.sample(3)

,date_publish,maintext,title,source_type,source_name,country,text,generated_text,topic
1363,2019-12-06 07:51:25,"Residents of Ryrkapiy, population 600, were sa...",Dozens of Polar Bears Descend Upon Russian Town,local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...","###Human:\nAs a neutral news analyst, assess t...",politics
775,2020-06-22 15:40:53,One of the first wind power stations in Russia...,Russia’s Coronavirus Recovery Plan Has No Spac...,local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...","###Human:\nAs a neutral news analyst, assess t...",politics
3534,2023-06-27 08:31:46,Russian mercenary chief Yevgeny Prigozhin flew...,Renegade Russian mercenary Prigozhin is in Bel...,international,reuters,russia,"As a neutral news analyst, assess the sentimen...","###Human:\nAs a neutral news analyst, assess t...",conflict


In [34]:
df.shape

(4000, 9)

How many has answers from LLaMa-2

In [35]:
df.generated_text[df.generated_text != ""].count()

4000

# Trimming to only Assistance Portion

In [36]:
def trim(text):
    pattern = "###Assistant:\n"
    matches = [match for match in re.finditer(pattern, text)]
    
    if matches is None:
        return ""
    
    try:
        return text[matches[0].end():].strip()
    except Exception:
        return None

In [37]:
print(trim(df.generated_text.iloc[0]))

1. Sentiment: Positive
    * Score: 0.8
    * Reason: The article conveys a positive sentiment, focusing on the progress made by Ukraine in retaking villages and the morale-boosting statements from President Zelensky.
2. Stance: Pro-Ukraine
    * Score: 0.9
    * Reason: The article takes a clear stance in favor of Ukraine, highlighting the courage and determination of its armed forces and their successful counter-offensive against Russian forces.


In [38]:
df.generated_text = df.generated_text.map(trim)

In [39]:
from src.llama2.parse_csv import ParseCSV

parser = ParseCSV()
df = parser(df)
df.sample(3)

,date_publish,maintext,title,source_type,source_name,country,text,generated_text,topic,sentiment_class,stance_class,sentiment_score,stance_score,sentiment_reason,stance_reason
511,2019-08-27 18:18:31,Putin was speaking after talks in Moscow with ...,"Russia, Turkey Agree Steps to Tackle Militants...",local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Positive\n * Score: 0.8\n ...,politics,positive,impartial,0.8,0.2,"The article presents a positive sentiment, as ...",The article provides an unbiased report on the...
2884,2023-04-17 07:49:38,Russia's Gazprombank has expanded its links wi...,Russia's Gazprombank deepens ties with Indian ...,international,reuters,russia,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Positive\n * Score: 0.7\n ...,politics,positive,impartial,0.7,0.0,The article discusses the expansion of ties be...,The article presents both Russia and India in ...
185,2023-07-12 11:02:36,"Elle Solomina is 36, but she says her life tru...",Trans advocates in Russia brace for proposed g...,international,reuters,russia,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Negative\n * Score: -0.8\n ...,politics,negative,against-russia,-0.8,-0.9,"The article presents a negative sentiment, foc...",The article portrays the proposed legislation ...


In [40]:
df[(df.country == "canada") & (df.generated_text != "")].source_type.value_counts()

Series([], Name: source_type, dtype: int64)

In [41]:
df.drop(columns=["text", "generated_text"], inplace=True)
df.columns

Index(['date_publish', 'maintext', 'title', 'source_type', 'source_name',
       'country', 'topic', 'sentiment_class', 'stance_class',
       'sentiment_score', 'stance_score', 'sentiment_reason', 'stance_reason'],
      dtype='object')

In [42]:
df.to_csv(r"D:\Studying\UoR\1. Data Mining\Final_Project\data\parsed\russia.csv", index=False)